In [1]:
from openai import OpenAI

In [2]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [3]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

/Users/crystalbravo/opt/anaconda3/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
index_name = "movie-questions"
model_name = "multi-qa-distilbert-cos-v1"
model = SentenceTransformer(model_name)

In [5]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 


In [6]:
def hybrid_search_with_rrf(query, k=60):

        query_vector = model.encode(query)
        keyword_query = {
        "bool": {
            "must": {
                 "multi_match": {
                     "query": query,
                        "fields": ["title", "summary", "genres"],
                        "type": "best_fields",
                        "boost": 0.5
                    }
                },
            }
        }
    
        knn_query = {
            "field": "title_summary_genres_vector",
            "query_vector": query_vector,
            "k": 20,
            "num_candidates": 100,
            "boost": 0.5
        }
        #Hybrid search: combining both text and vector search (at least evaluating it)

        knn_results = es_client.search(
            index=index_name,
            body={
                "knn": knn_query,
                "size": 10
            }
        )['hits']['hits']

        keyword_results = es_client.search(
            index=index_name,
            body={
                "query": keyword_query,
                "size": 10
            }
        )['hits']['hits']

        rrf_scores = {}
        # Calculate RRF using vector search results
        for rank, hit in enumerate(knn_results):
            doc_id = hit['_id']
            rrf_scores[doc_id] = compute_rrf(rank + 1, k)

        # Adding keyword search result scores
        for rank, hit in enumerate(keyword_results):
            doc_id = hit['_id']
            if doc_id in rrf_scores:
                rrf_scores[doc_id] += compute_rrf(rank + 1, k)
            else:
                rrf_scores[doc_id] = compute_rrf(rank + 1, k)

        # Sort RRF scores in descending order
        reranked_docs = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)

        # Document re-ranking (1 point)
        # Get top-K documents by the score
        final_results = []
        for doc_id, score in reranked_docs[:5]:
            doc = es_client.get(index=index_name, id=doc_id)
            final_results.append(doc['_source'])

        return final_results

def compute_rrf(self, rank, k=60):
    """ Our own implementation of the relevance score """
    return 1 / (k + rank)


In [7]:
prompt_template = """
You're a movie consault. Answer the QUESTION based on the CONTEXT from our movies database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
title: {title}               
year: {year}     
summary: {summary}     
short_summary: {short_summary}     
genres: {genres}     
imdb_id: {imdb_id}     
runtime: {runtime}     
youtube_trailer: {youtube_trailer}     
rating: {rating}     
movie_poster: {movie_poster}     
director: {director}     
writers: {writers}     
cast: {cast}  
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [8]:
def rag(query):
    search_results = hybrid_search_with_rrf(query)
    prompt = build_prompt(query, search_results)
    response = client.chat.completions.create(
        model= 'gemma2:2b',
        messages=[{"role": "user", "content": prompt}],
    )
    answer = response.choices[0].message.content
    return answer

In [9]:
#Modify the question to get the answer
question = 'Based on the IMDb rating, list top 5 movies.'


In [11]:
import os

In [15]:
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [17]:
rag(question)

'Based on the IMDb rating provided in your context, here are the top 5 movies:\n\n1. **The Disaster Artist:** 76\n2. **Split:** 73\n3. **The LEGO Batman Movie:** 73\n4. **The Last Movie Star:** 68\n5. **VHS Massacre:** 60 \n'